In [1]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.5/890.5 kB 22.2 MB/s eta 0:00:00


In [2]:
!pip install torch-fidelity

In [3]:
import os
import torch
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
from torchmetrics.image.fid import FrechetInceptionDistance
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder
import os
from PIL import Image
from tqdm import tqdm
from torchvision.utils import save_image
# local module imports
import config
from utils import save_checkpoint, load_checkpoint, save_some_examples
from UvU_Net_Generator import OuterUNet as Generator
from UvU_Discriminator import Discriminator

torch.backends.cudnn.benchmark = True

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


torch.Size([1, 3, 512, 512])


In [4]:
def train_fn(
    disc, gen, loader, opt_disc, opt_gen, l1_loss, bce, g_scaler, d_scaler,
):
    loop = tqdm(loader, leave=True)

    for idx, (x, y) in enumerate(loop):
        x = x.to(config.DEVICE)
        y = y.to(config.DEVICE)

        # Train Discriminator
        with torch.cuda.amp.autocast():
            y_fake = gen(x)
            D_real = disc(x, y)
            D_real_loss = bce(D_real, torch.ones_like(D_real))
            D_fake = disc(x, y_fake.detach())
            D_fake_loss = bce(D_fake, torch.zeros_like(D_fake))
            D_loss = (D_real_loss + D_fake_loss) / 2

        disc.zero_grad()
        d_scaler.scale(D_loss).backward()
        d_scaler.step(opt_disc)
        d_scaler.update()

        # Train generator
        with torch.cuda.amp.autocast():
            D_fake = disc(x, y_fake)
            G_fake_loss = bce(D_fake, torch.ones_like(D_fake))
            L1 = l1_loss(y_fake, y) * config.L1_LAMBDA
            G_loss = G_fake_loss + L1

        opt_gen.zero_grad()
        g_scaler.scale(G_loss).backward()
        g_scaler.step(opt_gen)
        g_scaler.update()

        if idx % 10 == 0:
            loop.set_postfix(
                D_real=torch.sigmoid(D_real).mean().item(),
                D_fake=torch.sigmoid(D_fake).mean().item(),
            )

In [5]:
!unzip -q Sample_dataset.zip

In [6]:
class PairedImageDataset(Dataset):
    def __init__(self, input_dir, target_dir, transform=None):
        self.input_dir = input_dir
        self.target_dir = target_dir
        self.input_images = sorted(os.listdir(input_dir))
        self.target_images = sorted(os.listdir(target_dir))
        self.transform = transform

        assert len(self.input_images) == len(self.target_images), "Mismatch between input and target images!"

    def __len__(self):
        return len(self.input_images)

    def __getitem__(self, idx):
        input_image_path = os.path.join(self.input_dir, self.input_images[idx])
        target_image_path = os.path.join(self.target_dir, self.target_images[idx])

        input_image = Image.open(input_image_path).convert("RGB")
        target_image = Image.open(target_image_path).convert("RGB")

        if self.transform:
            input_image = self.transform(input_image)
            target_image = self.transform(target_image)

        return input_image, target_image


input_dir = '/content/Sample_dataset/sobel_images'
target_dir = '/content/Sample_dataset/input_images'

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

dataset = PairedImageDataset(input_dir=input_dir, target_dir=target_dir, transform=transform)

# Split dataset into training and validation (80-20 split)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoader for training and validation
batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

# Example to iterate over the data loader
for batch_idx, (input_images, target_images) in enumerate(train_loader):
    print(f"Batch {batch_idx + 1}")
    print(f"Input batch size: {input_images.shape}")
    print(f"Target batch size: {target_images.shape}")


Batch 1
Input batch size: torch.Size([16, 3, 256, 256])
Target batch size: torch.Size([16, 3, 256, 256])
Batch 2
Input batch size: torch.Size([4, 3, 256, 256])
Target batch size: torch.Size([4, 3, 256, 256])


In [7]:
import os
import torch
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
from torchmetrics.image.fid import FrechetInceptionDistance
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Create directories
save_dir = '/content/drive/MyDrive/model_checkpoints/'  # Change this to your desired directory
os.makedirs(save_dir, exist_ok=True)
directory = '/content/evaluation/'
if not os.path.exists(directory):
    os.makedirs(directory)  # Create the directory if it doesn't exist

# Calculate classification metrics for the discriminator
def calculate_classification_metrics(disc, gen, val_loader, device):
    gen.eval()
    disc.eval()

    y_true = []
    y_pred = []

    with torch.no_grad():
        for x, y in val_loader:
            x = x.to(device)
            y = y.to(device)

            # Discriminator on real images
            # Assuming x is your input and y is the corresponding label for real images.
            batch_size = x.size(0)  # Dynamically get the current batch size

            # Discriminator on real images
            D_real = disc(x, y)
            real_preds = (torch.sigmoid(D_real) >= 0.5).cpu().numpy()  # Predictions for real images

            # Extend y_true with 1s for real images (1 = real)
            y_true.extend([1] * len(real_preds.flatten().tolist()))  # Ground truth for real images is 1
            y_pred.extend(real_preds.flatten().tolist())  # Extend predictions for real images

            # Generate fake images using the generator
            y_fake = gen(x)

            # Discriminator on fake images
            D_fake = disc(x, y_fake)
            fake_preds = (torch.sigmoid(D_fake) < 0.5).cpu().numpy()  # Predictions for fake images

            # Extend y_true with 0s for fake images (0 = fake)
            y_true.extend([0] * len(real_preds.flatten().tolist()))  # Ground truth for fake images is 0
            y_pred.extend(fake_preds.flatten().tolist())  # Extend predictions for fake images

    # Convert to numpy arrays for metrics
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    print(f"y_true size: {(y_true.shape)}, y_pred size: {(y_pred.shape)}")  # Check sizes here


    # Confusion matrix elements
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    # Accuracy
    accuracy = accuracy_score(y_true, y_pred)

    # True Positive Rate (TPR)
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0

    # False Positive Rate (FPR)
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0

    # True Negative Rate (TNR)
    tnr = tn / (tn + fp) if (tn + fp) > 0 else 0

    # False Negative Rate (FNR)
    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0

    # Precision
    precision = precision_score(y_true, y_pred)

    # Recall (equivalent to TPR)
    recall = recall_score(y_true, y_pred)

    # F1 Score
    f1 = f1_score(y_true, y_pred)

    print(f"Accuracy: {accuracy:.4f}, TPR: {tpr:.4f}, FPR: {fpr:.4f}, TNR: {tnr:.4f}, FNR: {fnr:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    return accuracy, tpr, fpr, tnr, fnr, precision, recall, f1

# FID Calculation (using torchmetrics)
def calculate_fid(gen, val_loader, device, fid):
    gen.eval()
    fid.reset()  # Reset FID metric before calculation
    with torch.no_grad():
        for x, y in val_loader:
            x = x.to(device)
            y_fake = gen(x)
            y_fake_tensor = y_fake.cpu().float()
            y_real_tensor = y.cpu().float()

            # Update FID with real and fake images
            fid.update(y_fake_tensor, real=False)
            fid.update(y_real_tensor, real=True)

    fid_score = fid.compute().item()  # Compute FID score
    print(f"FID: {fid_score}")
    return fid_score

# PSNR calculation
def calculate_psnr(gen, val_loader, device):
    gen.eval()
    total_psnr = 0.0
    num_batches = 0

    with torch.no_grad():
        for x, y in val_loader:
            x = x.to(device)
            y = y.to(device)
            y_fake = gen(x)

            mse = torch.mean((y - y_fake) ** 2)
            if mse == 0:
                psnr = 100
            else:
                psnr = 20 * torch.log10(1.0 / torch.sqrt(mse))

            total_psnr += psnr.item()
            num_batches += 1

    avg_psnr = total_psnr / num_batches
    print(f"PSNR: {avg_psnr:.4f}")
    return avg_psnr

# Plot metrics
def plot_metrics(metrics, num_epochs):
    epochs = list(range(1, num_epochs + 1))

    plt.figure(figsize=(18, 10))

    # Accuracy plot
    plt.subplot(3, 3, 1)
    plt.plot(epochs, metrics['accuracy'], label="Accuracy")
    plt.title("Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.grid(True)

    # TPR plot
    plt.subplot(3, 3, 2)
    plt.plot(epochs, metrics['tpr'], label="TPR")
    plt.title("True Positive Rate (TPR)")
    plt.xlabel("Epochs")
    plt.ylabel("TPR")
    plt.grid(True)

    # FPR plot
    plt.subplot(3, 3, 3)
    plt.plot(epochs, metrics['fpr'], label="FPR")
    plt.title("False Positive Rate (FPR)")
    plt.xlabel("Epochs")
    plt.ylabel("FPR")
    plt.grid(True)

    # TNR plot
    plt.subplot(3, 3, 4)
    plt.plot(epochs, metrics['tnr'], label="TNR")
    plt.title("True Negative Rate (TNR)")
    plt.xlabel("Epochs")
    plt.ylabel("TNR")
    plt.grid(True)

    # FNR plot
    plt.subplot(3, 3, 5)
    plt.plot(epochs, metrics['fnr'], label="FNR")
    plt.title("False Negative Rate (FNR)")
    plt.xlabel("Epochs")
    plt.ylabel("FNR")
    plt.grid(True)

    # Precision plot
    plt.subplot(3, 3, 6)
    plt.plot(epochs, metrics['precision'], label="Precision")
    plt.title("Precision")
    plt.xlabel("Epochs")
    plt.ylabel("Precision")
    plt.grid(True)

    # Recall plot
    plt.subplot(3, 3, 7)
    plt.plot(epochs, metrics['recall'], label="Recall")
    plt.title("Recall")
    plt.xlabel("Epochs")
    plt.ylabel("Recall")
    plt.grid(True)

    # F1 Score plot
    plt.subplot(3, 3, 8)
    plt.plot(epochs, metrics['f1'], label="F1 Score")
    plt.title("F1 Score")
    plt.xlabel("Epochs")
    plt.ylabel("F1 Score")
    plt.grid(True)

    # FID plot
    plt.subplot(3, 3, 9)
    plt.plot(epochs, metrics['fid'], label="FID")
    plt.title("Frechet Inception Distance (FID)")
    plt.xlabel("Epochs")
    plt.ylabel("FID")
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Main function
def main():
    disc = Discriminator(in_channels=3).to(config.DEVICE)
    gen = Generator(in_channels=3, features=64).to(config.DEVICE)
    opt_disc = optim.Adam(disc.parameters(), lr=config.LEARNING_RATE, betas=(0.5, 0.999))
    opt_gen = optim.Adam(gen.parameters(), lr=config.LEARNING_RATE, betas=(0.5, 0.999))
    BCE = nn.BCEWithLogitsLoss()
    L1_LOSS = nn.L1Loss()

    if config.LOAD_MODEL:
        load_checkpoint(config.CHECKPOINT_GEN, gen, opt_gen, config.LEARNING_RATE)
        load_checkpoint(config.CHECKPOINT_DISC, disc, opt_disc, config.LEARNING_RATE)

    g_scaler = torch.cuda.amp.GradScaler()
    d_scaler = torch.cuda.amp.GradScaler()

    # Initialize metric storage
    metrics = {
        'accuracy': [],
        'tpr': [],
        'fpr': [],
        'tnr': [],
        'fnr': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'fid': [],
        'psnr': []
    }

    # Initialize FID object from torchmetrics
    fid = FrechetInceptionDistance(normalize=True).to('cpu')
    for epoch in range(config.NUM_EPOCHS):
        train_fn(disc, gen, train_loader, opt_disc, opt_gen, L1_LOSS, BCE, g_scaler, d_scaler)

        # Calculate metrics every epoch
        accuracy, tpr, fpr, tnr, fnr, precision, recall, f1 = calculate_classification_metrics(disc, gen, val_loader, config.DEVICE)
        fid_score = calculate_fid(gen, val_loader, config.DEVICE, fid)
        psnr_score = calculate_psnr(gen, val_loader, config.DEVICE)

        # Store metrics for plotting later
        metrics['accuracy'].append(accuracy)
        metrics['tpr'].append(tpr)
        metrics['fpr'].append(fpr)
        metrics['tnr'].append(tnr)
        metrics['fnr'].append(fnr)
        metrics['precision'].append(precision)
        metrics['recall'].append(recall)
        metrics['f1'].append(f1)
        metrics['fid'].append(fid_score)
        metrics['psnr'].append(psnr_score)

        print(f"Epoch {epoch} - Metrics: Accuracy: {accuracy:.4f}, TPR: {tpr:.4f}, FPR: {fpr:.4f}, FID: {fid_score:.4f}, PSNR: {psnr_score:.4f}")

        # Save model checkpoints every 50 epochs
        if epoch % 50 == 49:
            print(f"Saving model at epoch {epoch}")
            torch.save({
                'epoch': epoch,
                'generator_state_dict': gen.state_dict(),
                'optimizer_gen_state_dict': opt_gen.state_dict(),
            }, f"{save_dir}generator_epoch_{epoch}.pth")

            torch.save({
                'epoch': epoch,
                'discriminator_state_dict': disc.state_dict(),
                'optimizer_disc_state_dict': opt_disc.state_dict(),
            }, f"{save_dir}discriminator_epoch_{epoch}.pth")

            save_some_examples(gen, val_loader, epoch, folder="evaluation")

    # Plot metrics after training is complete
    plot_metrics(metrics, config.NUM_EPOCHS)


if __name__ == "__main__":
    main()


<ipython-input-7-cdde3062e277>:230: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  g_scaler = torch.cuda.amp.GradScaler()
<ipython-input-7-cdde3062e277>:231: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  d_scaler = torch.cuda.amp.GradScaler()
Downloading: "https://github.com/toshas/torch-fidelity/releases/download/v0.2.0/weights-inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/weights-inception-2015-12-05-6726825d.pth
100%|██████████| 91.2M/91.2M [00:00<00:00, 398MB/s]
  0%|          | 0/2 [00:00<?, ?it/s]<ipython-input-4-6490c8d8cca4>:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
<ipython-input-4-6490c8d8cca4>:25: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please u

y_true size: (9000,), y_pred size: (9000,)
Accuracy: 1.0000, TPR: 1.0000, FPR: 0.0000, TNR: 1.0000, FNR: 0.0000
Precision: 1.0000, Recall: 1.0000, F1 Score: 1.0000
FID: 455.6948547363281
PSNR: 6.4829
Epoch 0 - Metrics: Accuracy: 1.0000, TPR: 1.0000, FPR: 0.0000, FID: 455.6949, PSNR: 6.4829


  0%|          | 0/2 [00:00<?, ?it/s]<ipython-input-4-6490c8d8cca4>:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
<ipython-input-4-6490c8d8cca4>:25: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 2/2 [00:00<00:00,  4.66it/s, D_fake=0.478, D_real=0.541]


y_true size: (9000,), y_pred size: (9000,)
Accuracy: 0.3703, TPR: 0.7407, FPR: 1.0000, TNR: 0.0000, FNR: 0.2593
Precision: 0.4255, Recall: 0.7407, F1 Score: 0.5405
FID: 455.6948547363281
PSNR: 6.4829
Epoch 1 - Metrics: Accuracy: 0.3703, TPR: 0.7407, FPR: 1.0000, FID: 455.6949, PSNR: 6.4829


  0%|          | 0/2 [00:00<?, ?it/s]<ipython-input-4-6490c8d8cca4>:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
<ipython-input-4-6490c8d8cca4>:25: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 2/2 [00:00<00:00,  5.76it/s, D_fake=0.308, D_real=0.585]


...

  0%|          | 0/2 [00:00<?, ?it/s]<ipython-input-4-6490c8d8cca4>:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
<ipython-input-4-6490c8d8cca4>:25: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 2/2 [00:00<00:00,  5.01it/s, D_fake=0.0377, D_real=0.986]


y_true size: (9000,), y_pred size: (9000,)
Accuracy: 0.4881, TPR: 0.9762, FPR: 1.0000, TNR: 0.0000, FNR: 0.0238
Precision: 0.4940, Recall: 0.9762, F1 Score: 0.6560
FID: 526.5516967773438
PSNR: 9.8377
Epoch 357 - Metrics: Accuracy: 0.4881, TPR: 0.9762, FPR: 1.0000, FID: 526.5517, PSNR: 9.8377


  0%|          | 0/2 [00:00<?, ?it/s]<ipython-input-4-6490c8d8cca4>:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
<ipython-input-4-6490c8d8cca4>:25: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 2/2 [00:00<00:00,  4.97it/s, D_fake=0.0179, D_real=0.966]


y_true size: (9000,), y_pred size: (9000,)
Accuracy: 0.4972, TPR: 0.9944, FPR: 1.0000, TNR: 0.0000, FNR: 0.0056
Precision: 0.4986, Recall: 0.9944, F1 Score: 0.6642


KeyboardInterrupt: 